# Data Cleaning Error Log for APS Data
### Run Script in Jupyter and scroll to the end to get full error log

In [1]:
import pandas as pd
from IPython.display import display, HTML
from datetime import datetime 
import re


from spellchecker import SpellChecker
spell = SpellChecker()

spell.word_frequency.load_text_file('./streetdict.txt')
spell.word_frequency.load_text_file('./customdict.txt')

pd.options.mode.chained_assignment = None  # default='warn'

#### Import Excel File
##### Make sure excel is in the same directory as the project, is named 'accessible-pedestrian-signals.xlsx' and has 3 columns tittled "Location", "Borough" and "Date Installed" 

In [2]:
# Read excel file
df = pd.read_excel('accessible-pedestrian-signals.xlsx')
# Match pandas dataframe index with Excel row
df.index = df.index + 2
# print DataFrame head
df.head()

,Location,Borough,Date Installed
2,10th Avenue and 73rd Street,Brooklyn,2020-07-28 00:00:00
3,3rd Avenue and 16th Streett,Brooklyn,2021-05-25 00:00:00
4,3rd Avenue and 33rd Street,Brooklyn,2015-10-23 00:00:00
5,3rd Avenue and 59th Street,Brooklyn,2020-10-10 00:00:00
6,3rd Avenue and 9th Street,Brooklyn,2020-01-14 00:00:00


---
## Rows with missing data
#### Will be empty if no data is missing

In [3]:
null_errors = df[df.isna().any(axis=1)]
null_errors['Error'] =  'Missing Data'
null_errors

,Location,Borough,Date Installed,Error
7,4th Avenue and 240' South of 3rd Street (mid-b...,NaN,2019-04-11 00:00:00,Missing Data
29,86th Street and Bay 34th Street,Brooklyn,NaN,Missing Data
30,NaN,Brooklyn,2019-04-16 00:00:00,Missing Data
38,NaN,Brooklyn,2022-04-29 00:00:00,Missing Data


---
## Rows with duplicate data
#### Will be empty if no data is duplicated

In [4]:
duplicate_data = df[df.duplicated()] 
duplicate_data['Error'] = ' Duplicate Data'
duplicate_data

,Location,Borough,Date Installed,Error


---
## Date Column Error Checking
#### Return all entries that have an invalid date


In [5]:
#Remove missing rows
df2 = df.dropna(subset = ['Date Installed'])

date_errors = df2['Date Installed'] = pd.to_datetime(df2['Date Installed'], errors='coerce')
date_errors = df2.loc[df2['Date Installed'].isnull()]
date_errors['Error'] = 'Date is invalid'

date_errors

,Location,Borough,Date Installed,Error
10,4th Avenue and 44th Street,Brooklyn,NaT,Date is invalid


---
## Borough Column Error Checking
#### Return all entries that have an invalid borough

In [6]:
#Remove missing rows
df3 = df.dropna(subset = ['Borough'])

borough_errors = df3.loc[(df3['Borough'] != 'Brooklyn') & (df3['Borough'] != 'Queens') 
       & (df3['Borough'] != 'the Bronx') & (df3['Borough'] != 'Bronx')
                       & (df3['Borough'] != 'Manhattan') & (df3['Borough'] != 'Staten Island')]

borough_errors['Error'] = 'Borough spelling, capitalization or type'

borough_errors

,Location,Borough,Date Installed,Error


---
## Location Column Error Checking

### Abbreviations checked for consistency:
#### St (Street or Saint), Ave (Avenue), Blvd (Boulevard), Dr (Drive), Expy (Expressway)
#### Pkwy (Parkway), Pl(Place), Rd (Road), TER (Terrace)

In [7]:
# Returns the abbreviation contained in the abbrev_errors dataframe
def findabbrev(misspelled):
    list2 =['ST', 'ST.','AVE', 'BLVD','DR', 'EXPY', 'PKWY', 'PL','RD', "TER" ]
    abbrevlist=[]
    for name in misspelled:
        name = name.upper()
        if name in list2:
            abbrevlist.append(name)
    return abbrevlist

#Create 'abbrev_errors' dataframe with data that contains abbreviations
#Regex checks if word is by itself, '(?i)' makes the search case insensitive
abbrev_errors = df[df['Location'].str.contains(r"(?i)\bSt\b|\bAve\b|\bBlvd\b|\bDr\b|\bExpy\b|\bPkwy\b|\bPl\b|\bRd\b|\bTer\b")== True]
abbrev_errors['Abbreviation'] = abbrev_errors['Location'].str.split().apply(findabbrev)
abbrev_errors['Error']= 'Contains abbreviation:  ' + abbrev_errors['Abbreviation'].astype(str)
abbrev_errors.drop(columns =['Abbreviation'], axis=1, inplace=True)

abbrev_errors

,Location,Borough,Date Installed,Error
232,Hanson Place and St. Felix Street,Brooklyn,2015-05-13 00:00:00,Contains abbreviation: ['ST.']
317,Rockaway Pkwy and Brookdale Hospital entrance ...,Brooklyn,2016-10-21 00:00:00,Contains abbreviation: ['PKWY']
320,Rogers Avenue and St Johns Place,Brooklyn,2019-09-10 00:00:00,Contains abbreviation: ['ST']
588,St James Place and James Street,Manhattan,2013-11-20 00:00:00,Contains abbreviation: ['ST']
589,St Nicholas Avenue and Pedestrian Crossing bet...,Manhattan,2017-05-15 00:00:00,Contains abbreviation: ['ST']
590,St Nicholas Avenue and West 125th Street,Manhattan,2016-10-21 00:00:00,Contains abbreviation: ['ST']
591,St Nicholas Avenue and West 156th Street,Manhattan,2017-05-12 00:00:00,Contains abbreviation: ['ST']
592,St Nicholas Avenue and West 177th Street,Manhattan,2022-05-20 00:00:00,Contains abbreviation: ['ST']
593,St Nicholas Avenue and West 183rd Street,Manhattan,2022-06-24 00:00:00,Contains abbreviation: ['ST']
612,West 125th Street with St Clair Place and West...,Manhattan,2022-02-18 00:00:00,Contains abbreviation: ['ST']


---
## Check Location Column for Street Spelling
#### Prepping the data for Error Checking

In [8]:
#Remove missing rows
LocationSpellingErrors = df.dropna(subset = ['Location'])

#Replace special characters with whitespace in order to extract words
LocationSpellingErrors['Location'] = LocationSpellingErrors['Location'].str.replace('(',' ', regex=True)
LocationSpellingErrors['Location'] = LocationSpellingErrors['Location'].str.replace(')',' ', regex=True)
LocationSpellingErrors['Location'] = LocationSpellingErrors['Location'].str.replace('-',' ', regex=True)
LocationSpellingErrors['Location'] = LocationSpellingErrors['Location'].str.replace(',',' ', regex=True)
LocationSpellingErrors['Location'] = LocationSpellingErrors['Location'].str.replace("’",' ', regex=True)
LocationSpellingErrors['Location'] = LocationSpellingErrors['Location'].str.replace("'",' ', regex=True)
LocationSpellingErrors['Location'] = LocationSpellingErrors['Location'].str.replace("/",' ', regex=True)
LocationSpellingErrors['Location'] = LocationSpellingErrors['Location'].str.replace(".",' ', regex=True)

#Split Location Column into Substrings
LocationSpellingErrors['Split_words'] = LocationSpellingErrors['Location'].str.split()

In [9]:
#Store rows with data type error in 'LocationSpellingErrors' dataframe
LocationTypeError = LocationSpellingErrors[LocationSpellingErrors['Split_words'].isna()]
LocationTypeError['Error']= 'Location entry not valid'
LocationTypeError.drop(columns =['Split_words'], axis=1, inplace=True)
LocationSpellingErrors = LocationSpellingErrors.dropna(subset = ['Split_words'])

#Store rows with insufficient words in 'LocationLengthError' dataframe
#Column contains less than 3 words
LocationLengthError = LocationSpellingErrors[LocationSpellingErrors['Split_words'].map(lambda d: len(d)) < 4]
LocationLengthError['Error']= 'Location entry not complete'
LocationLengthError.drop(columns =['Split_words'], axis=1, inplace=True)

In [10]:
# Use PySpellChecker library to find misspelt words
def incorrectlist(stlist):
    return spell.unknown(stlist)

# Use PySpellChecker library to find potential corrections to misspelt words
def correctlist(misspelled):
    correctwords=[]
    for word in misspelled:
        correctwords.append(spell.correction(word))
        return correctwords


LocationSpellingErrors['Incorrect Words'] = LocationSpellingErrors['Split_words'].apply(incorrectlist).tolist()
LocationSpellingErrors = LocationSpellingErrors[LocationSpellingErrors['Incorrect Words'].map(lambda d: len(d)) > 0]
LocationSpellingErrors['Potential word'] = LocationSpellingErrors['Incorrect Words'].apply(correctlist).tolist()
LocationSpellingErrors["Error"] = "Location Spelling Error: " + LocationSpellingErrors['Incorrect Words'].astype(str)+ '\n' + "Did you mean: " + LocationSpellingErrors['Potential word'].astype(str) + '?'
LocationSpellingErrors.drop(columns =['Split_words', 'Incorrect Words','Potential word'], axis=1, inplace=True)
print("Location Spelling Errors")
LocationSpellingErrors

Location Spelling Errors


,Location,Borough,Date Installed,Error
3,3rd Avenue and 16th Streett,Brooklyn,2021-05-25 00:00:00,Location Spelling Error: {'streett'}\nDid you ...
17,4th Avenue and Mrospect Avenue,Brooklyn,2022-07-12 00:00:00,Location Spelling Error: {'mrospect'}\nDid you...
429,Avenue of the Americas and 23rd Street Selis ...,Manhattan,2004-05-25 00:00:00,Location Spelling Error: {'selis'}\nDid you me...
450,Broadway and West 133th Street,Manhattan,2019-05-06 00:00:00,Location Spelling Error: {'133th'}\nDid you me...
795,Jamaica Avenue and 189th Stret,Queens,2020-03-24 00:00:00,Location Spelling Error: {'stret'}\nDid you me...
925,Woodhaven Boulevard NB and Long Island Epres...,Queens,2012-03-14 00:00:00,Location Spelling Error: {'epressway'}\nDid yo...
957,Hylan Boulevard and Huegenot Avenue,Staten Island,2019-03-13 00:00:00,Location Spelling Error: {'huegenot'}\nDid you...


---
## Full Error Log
### Errors are also exported in an excel file in the project directory
#### You can remove a specific error check by deleting it from the 'frames' list below


In [11]:
# Concat all error dfs, comment out any errors you want to exlcude from final error log
frames = [null_errors, 
          duplicate_data, 
          borough_errors, 
          date_errors, 
          LocationTypeError,
          LocationLengthError, 
          LocationSpellingErrors, 
          abbrev_errors]

result = pd.concat(frames)

# Sort errors by row index
result.sort_index(inplace=True)
#Label row
result.index.rename('Row', inplace=True)


if len(result.index) == 0:
    print('No Errors Found')
else:
#     Restore values of original dataframe that may have been converted to 'NA'
    result["Location"] = df['Location']
    result["Borough"] = df['Borough']
    result["Date Installed"] = df['Date Installed']
    print(str(len(result.index)) +' Errors Found')
    display( HTML( result.to_html().replace("\\n","<br>") ) )
#     Export Errors in an excel file
    result.to_excel('errors.xlsx')

29 Errors Found


,Location,Borough,Date Installed,Error
Row,,,,
3,3rd Avenue and 16th Streett,Brooklyn,2021-05-25 00:00:00,Location Spelling Error: {'streett'}Did you mean: ['street']?
7,4th Avenue and 240' South of 3rd Street (mid-block),NaN,2019-04-11 00:00:00,Missing Data
10,4th Avenue and 44th Street,Brooklyn,10/21/2015564664,Date is invalid
17,4th Avenue and Mrospect Avenue,Brooklyn,2022-07-12 00:00:00,Location Spelling Error: {'mrospect'}Did you mean: ['prospect']?
29,86th Street and Bay 34th Street,Brooklyn,NaN,Missing Data
30,NaN,Brooklyn,2019-04-16 00:00:00,Missing Data
38,NaN,Brooklyn,2022-04-29 00:00:00,Missing Data
232,Hanson Place and St. Felix Street,Brooklyn,2015-05-13 00:00:00,Contains abbreviation: ['ST.']
317,Rockaway Pkwy and Brookdale Hospital entrance (w of Linden Blvd),Brooklyn,2016-10-21 00:00:00,Contains abbreviation: ['PKWY']
